In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import csv
from requests import post
import numpy as np

In [7]:
steam_data =pd.read_csv(r"C:\Users\valdi\OneDrive\Documents\Springboard\steam.csv")
steam_data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [8]:
steam_data.shape

(27075, 18)

In [9]:
steam_data.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

The next step will be dropping data that is unneeded for the machine learning program
which are "required_age", "english", "achievements", "owners", "steamspy_tags",  and "price"
as these columns either do not reflect traits of trending games or are redundant. Although, appid is not vital for the machine learning program, but it is in order to merge the steamspy_tag_data.csv file   

In [10]:
steam_data=steam_data.drop(['english','required_age','achievements','owners','price','steamspy_tags' ], axis=1)
steam_data.columns

Index(['appid', 'name', 'release_date', 'developer', 'publisher', 'platforms',
       'categories', 'genres', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime'],
      dtype='object')

COLUMN DESCRIPTION:
name-tilte of the game
release_date- when the game was released
developer-developer of the game
publisher-publisher of the game
platforms- what platforms these games were on
categories- determines the type of game
genres- determines the sub-type of the game 
positive_ratings- ratings that reflect positve reviews of the game 
negative_ratings- rating that reflect negative reviews of the game
average_playtime- the average amount of time a player spent on the game
median_playtime- the midpoint of this dataset on playtime

In [11]:
steam_data.head()

,appid,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,average_playtime,median_playtime
0,10,Counter-Strike,2000-11-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,124534,3339,17612,317
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,3318,633,277,62
2,30,Day of Defeat,2003-05-01,Valve,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,3416,398,187,34
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,1273,267,258,184
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,5250,288,624,415


In [12]:
steam_data.describe(include = 'all')

,appid,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,average_playtime,median_playtime
count,2.707500e+04,27075,27075,27075,27075,27075,27075,27075,2.707500e+04,27075.000000,27075.000000,27075.00000
unique,NaN,27033,2619,17113,14354,7,3333,1552,NaN,NaN,NaN,NaN
top,NaN,Dark Matter,2018-07-13,Choice of Games,Big Fish Games,windows,Single-player,Action;Indie,NaN,NaN,NaN,NaN
freq,NaN,3,64,94,212,18398,6110,1852,NaN,NaN,NaN,NaN
mean,5.962035e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000559e+03,211.027147,149.804949,146.05603
std,2.508942e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.898872e+04,4284.938531,1827.038141,2353.88008
min,1.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.00000
25%,4.012300e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000e+00,2.000000,0.000000,0.00000
50%,5.990700e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000e+01,9.000000,0.000000,0.00000
75%,7.987600e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.260000e+02,42.000000,0.000000,0.00000


The reason why NaN seen so often in first few colmuns is due to the fact that 
columns "name", 'release_date;, 'devleoper', 'publisher', 'platforms', 'categories'
and 'genres' are considered to be categorical data. 

In [13]:
print(f'name: {steam_data.name.dtype}')
print(f'release_date: {steam_data.release_date.dtype}')
print(f'developer: {steam_data.developer.dtype}')
print(f'publisher: {steam_data.publisher.dtype}')
print(f'platforms: {steam_data.platforms.dtype}')
print(f'categories: {steam_data.categories.dtype}')
print(f'genres: {steam_data.genres.dtype}')
print(f'positive_ratings: {steam_data.positive_ratings.dtype}')
print(f'negative_ratings: {steam_data.negative_ratings.dtype}')
print(f'average_playtime: {steam_data.average_playtime.dtype}')
print(f'median_playtime: {steam_data.median_playtime.dtype}')

name: object
release_date: object
developer: object
publisher: object
platforms: object
categories: object
genres: object
positive_ratings: int64
negative_ratings: int64
average_playtime: int64
median_playtime: int64


In [14]:
print(f'name: {steam_data.name.nunique()}')
print(f'release_date: {steam_data.release_date.nunique()}')
print(f'developer: {steam_data.developer.nunique()}')
print(f'publisher: {steam_data.publisher.nunique()}')
print(f'platforms: {steam_data.platforms.nunique()}')
print(f'categories: {steam_data.categories.nunique()}')
print(f'genres: {steam_data.genres.nunique()}')
print(f'positive_ratings: {steam_data.positive_ratings.nunique()}')
print(f'negative_ratings: {steam_data.negative_ratings.nunique()}')
print(f'average_playtime: {steam_data.average_playtime.nunique()}')
print(f'median_playtime: {steam_data.median_playtime.nunique()}')
#This is done so the output may reflect on how the data was measured as well as noticing which colmuns may have
#repeating/missing rows

name: 27033
release_date: 2619
developer: 17113
publisher: 14354
platforms: 7
categories: 3333
genres: 1552
positive_ratings: 2800
negative_ratings: 1492
average_playtime: 1345
median_playtime: 1312


In [15]:
steam_data.isnull().sum()

appid               0
name                0
release_date        0
developer           0
publisher           0
platforms           0
categories          0
genres              0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
dtype: int64

In [16]:
duplicate=steam_data[steam_data.duplicated()]
print(duplicate)
#An empty dataframe is produced, meaning that there are no values in this dataframe, therfore it is dafe to conclude there are 
#no duplicated values

Empty DataFrame
Columns: [appid, name, release_date, developer, publisher, platforms, categories, genres, positive_ratings, negative_ratings, average_playtime, median_playtime]
Index: []


In [17]:
print(steam_data.positive_ratings)

0        124534
1          3318
2          3416
3          1273
4          5250
          ...  
27070         3
27071         8
27072         0
27073         2
27074         4
Name: positive_ratings, Length: 27075, dtype: int64


In [18]:
steam_data['genres'].unique()

array(['Action', 'Action;Free to Play', 'Action;Free to Play;Strategy',
       ...,
       'Action;Adventure;Indie;Massively Multiplayer;RPG;Strategy;Early Access',
       'Action;Adventure;Casual;Free to Play;Indie;RPG;Simulation;Sports;Strategy',
       'Casual;Free to Play;Massively Multiplayer;RPG;Early Access'],
      dtype=object)

In [19]:
df= steam_data['genres'].str.split(';', expand= True)


In [20]:
df['Name']= steam_data['name']
df.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 'Name'], dtype='object')

In [21]:
df_one=pd.get_dummies(df, prefix= " ", prefix_sep= " ")
print(df_one)

         Accounting    Action    Adventure    Animation & Modeling  \
0                 0         1            0                       0   
1                 0         1            0                       0   
2                 0         1            0                       0   
3                 0         1            0                       0   
4                 0         1            0                       0   
...             ...       ...          ...                     ...   
27070             0         0            1                       0   
27071             0         1            0                       0   
27072             0         1            0                       0   
27073             0         0            1                       0   
27074             0         0            1                       0   

         Audio Production    Casual    Design & Illustration    Early Access  \
0                       0         0                        0               0   

In [22]:
steamspy_tag_data = pd.read_csv(r"steamspy_tag_data.csv")
steamspy_tag_data.head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
steamspy_tag_data.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of        appid  1980s  1990s   2.5d     2d  2d_fighter  360_video     3d  \
0      False  False  False  False  False       False      False  False   
1      False  False  False  False  False       False      False  False   
2      False  False  False  False  False       False      False  False   
3      False  False  False  False  False       False      False  False   
4      False  False  False  False  False       False      False  False   
...      ...    ...    ...    ...    ...         ...        ...    ...   
29017  False  False  False  False  False       False      False  False   
29018  False  False  False  False  False       False      False  False   
29019  False  False  False  False  False       False      False  False   
29020  False  False  False  False  False       False      False  False   
29021  False  False  False  False  False       False      False  False   

       3d_platformer  3d_vision  ...  warhammer_4

In [24]:
duplicate_steamspy=steamspy_tag_data[steamspy_tag_data.duplicated()]
print(duplicate_steamspy)

Empty DataFrame
Columns: [appid, 1980s, 1990s, 2.5d, 2d, 2d_fighter, 360_video, 3d, 3d_platformer, 3d_vision, 4_player_local, 4x, 6dof, atv, abstract, action, action_rpg, action_adventure, addictive, adventure, agriculture, aliens, alternate_history, america, animation_&_modeling, anime, arcade, arena_shooter, artificial_intelligence, assassin, asynchronous_multiplayer, atmospheric, audio_production, bmx, base_building, baseball, based_on_a_novel, basketball, batman, battle_royale, beat_em_up, beautiful, benchmark, bikes, blood, board_game, bowling, building, bullet_hell, bullet_time, crpg, capitalism, card_game, cartoon, cartoony, casual, cats, character_action_game, character_customization, chess, choices_matter, choose_your_own_adventure, cinematic, city_builder, class_based, classic, clicker, co_op, co_op_campaign, cold_war, colorful, comedy, comic_book, competitive, conspiracy, controller, conversation, crafting, crime, crowdfunded, cult_classic, cute, cyberpunk, cycling, dark, da

In [26]:
steamspy_tag_data.columns

Index(['appid', '1980s', '1990s', '2.5d', '2d', '2d_fighter', '360_video',
       '3d', '3d_platformer', '3d_vision',
       ...
       'warhammer_40k', 'web_publishing', 'werewolves', 'western', 'word_game',
       'world_war_i', 'world_war_ii', 'wrestling', 'zombies', 'e_sports'],
      dtype='object', length=372)

In [25]:
merged_steamdata=steam_data.merge(steamspy_tag_data, how='right')
merged_steamdata.head()


,appid,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,Counter-Strike,2000-11-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,124534.0,3339.0,...,0,0,0,0,0,0,0,0,0,550
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,3318.0,633.0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003-05-01,Valve,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,3416.0,398.0,...,0,0,0,0,0,5,122,0,0,0
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,1273.0,267.0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,5250.0,288.0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
merged_steamdata.describe(include='all')

,appid,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
count,2.902200e+04,27075,27075,27075,27075,27075,27075,27075,2.707500e+04,27075.000000,...,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000
unique,NaN,27033,2619,17113,14354,7,3333,1552,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Dark Matter,2018-07-13,Choice of Games,Big Fish Games,windows,Single-player,Action;Indie,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,64,94,212,18398,6110,1852,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5.952577e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000559e+03,211.027147,...,0.134140,0.049962,0.017573,0.158742,0.023637,0.092034,0.761698,0.016470,3.314382,0.574426
std,2.521478e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.898872e+04,4284.938531,...,5.722873,1.335872,0.934081,7.371732,1.002216,5.736370,24.977839,0.892563,104.515689,56.920088
min,1.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.997825e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000e+00,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.994700e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000e+01,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.987275e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.260000e+02,42.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
merged_steamdata.columns

Index(['appid', 'name', 'release_date', 'developer', 'publisher', 'platforms',
       'categories', 'genres', 'positive_ratings', 'negative_ratings',
       ...
       'warhammer_40k', 'web_publishing', 'werewolves', 'western', 'word_game',
       'world_war_i', 'world_war_ii', 'wrestling', 'zombies', 'e_sports'],
      dtype='object', length=383)

To summarize the purpose of this notebook is to start the early steps of creating a Machine Learning program designed to not only identify top trending games and generes but as well as future trends. The first step is collecting data from a reputaible source, as stated in the proposal this data orignates from steam. Once this step is performed, the data was grossly-analyzed. Just understanding, the type of data how many columns and rows, if there any missing or strange values, basic statistics, and/or how the data was recorded(binary or etc). The "genere" aspect of the data was converted to a binary system in order to make the data easier to manipulate. I have also added a new dataframe that consist of data that describes the amount of upvotes each genre of game which that table has been merged with the steam_data dataframe. In the next journal, this data will be used to create a histogram. The histogram will visualize least and most popular generes.